In [ ]:
import os
import io
import boto3
from torch.autograd import Variable
from torch import from_numpy
import matplotlib.pyplot as plt
import sagemaker
from sagemaker.pytorch import PyTorch


In [ ]:
n_epochs = 300
learning_rate = 0.01

## Run Training in SageMaker

In [ ]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-iris/trainingdata'

role = sagemaker.get_execution_role()
inputs = 's3://{}/{}'.format(bucket, prefix)

In [ ]:
estimator = PyTorch(entry_point="iris.py",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.m4.xlarge',
                    hyperparameters={
                        'epochs': n_epochs,
                        'lr': learning_rate
                    })

In [ ]:
estimator.fit({'training': inputs})

## Deploy

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
s3 = boto3.resource("s3")
databucket = s3.Bucket(bucket)
test_data_key = "{}/iris_test_data.npy".format(prefix)
test_target_key = "{}/iris_test_target.npy".format(prefix)
X_test = None
y_test = None
with io.BytesIO(databucket.Object(test_data_key).get()["Body"].read()) as f:
    f.seek(0)
    X_test = np.load(f)
with io.BytesIO(databucket.Object(test_target_key).get()["Body"].read()) as f:
    f.seek(0)
    y_test = np.load(f)


In [ ]:
for i in range(response.shape[0]):
    print("Target = {}, Prediction = {}, ({})".format(
            y_test[i], 
            np.argmax(response[i]),
            "Match" if y_test[i] == np.argmax(response[i]) else "No Match"
        )
    )